## One phase flow.
### pezoflow equation.

$$\frac{\partial \rho_o m}{\partial t} + \frac{\partial}{\partial x}(\rho_o W_o)=0$$
$$W_o=-\frac{k}{\mu_o}\frac{\partial p}{\partial x}$$
$$m(p)=m_0+\beta_r*(p-p_0)$$
$$\rho_{o}(p) = \rho_{o0}(1 + \beta_{o}*(p-p_0))$$

$$x\in[0,L]$$
$$t\in[0,T]$$

*Initial and boundary conditions*
$$p(x, t=0)=p_0$$
$$p(x=0, t)=p_{left}$$
$$p(x=L, t)=p_0$$

1. $$ m \rho_{o0} \beta_{o}\frac{\partial p}{\partial t} + \rho_o \beta_r \frac{\partial p}{\partial t}-\frac{\partial}{\partial x}(\rho_o \frac{k}{\mu_o}\frac{\partial p}{\partial x})=0$$

2. $$ \frac{\partial p}{\partial t} (m \rho_{o0} \beta_{o} + \rho_o \beta_r) -\frac{\partial}{\partial x}(\rho_o \frac{k}{\mu_o}\frac{\partial p}{\partial x})=0$$

In [55]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import newton_krylov, fsolve
from scipy.interpolate import interp1d
from sol_plot import solution_plot
from copy import copy

def expit(x):
    return 1 / (1 + np.exp(-x))

oil_visc = lambda mu_h, mu_o, glad, grad, G: (mu_h-mu_o) * expit (glad * (-grad + G)) + mu_o


In [3]:
class OnePhase():
    def __init__(self,
                 mu_h,
                 mu_o,
                 glad,
                 G,
                 k,
                 m0,
                 beta_r,
                 rho_o0,
                 beta_o,
                 t_0,
                 tt,
                 T,
                 nt,
                 pow_n,
                 x_0,
                 L,
                 L0,
                 S,
                 nx,
                 p_left,
                 p_right,
                 mu_type,
                 Q=None) -> None:
        self.mu_h = mu_h
        self.mu_o = mu_o
        self.glad = glad
        self.G = G
        self.k = k
        self.m0 = m0
        self.Q = Q
        self.beta_r = beta_r
        self.rho_o0 = rho_o0
        self.beta_o = beta_o
        self.t_0 = t_0
        self.tt = tt,
        self.T = T
        self.nt = nt
        self.pow_n = pow_n
        self.x_0 = x_0
        self.L = L
        self.S = S
        self.nx = nx
        self.p_left = p_left
        self.p_right = p_right
        self.time = np.linspace(t_0, T, int(nt), dtype=np.float64)
        self.tt = self.time.flat[np.abs(self.time - tt).argmin()]
        self.dt = self.time[1]-self.time[0]
        self.x = np.linspace(0, L, nx, dtype=np.float64)
        self.dx = self.x[1]-self.x[0]
        self.mu_oil_arr = []
        self.grad_p = []
        self.grad_mu = []
        # self.p_0 = p_right
        self.mu_type = mu_type
        self.mu_stop = np.zeros_like(self.x)
        self.p_0 = lambda coord : np.where(coord<=L/L0, np.ones_like(coord)*self.p_left, np.ones_like(coord)*self.p_right)

    def m(self, p, x):
        return self.m0 + self.beta_r*(p-self.p_0(x))

    def rho_o(self, p, x):
        return self.rho_o0*(1+self.beta_o*(p-self.p_0(x)))

    def mu_oil(self, grad, grad_old, mu_stop):
        grad = abs(grad)
        grad_old = abs(grad_old)
        mu = (self.mu_h-self.mu_o) * expit (self.glad * (-grad + self.G)) + self.mu_o
        if self.mu_type=='mu_2side':
            mu_new = (self.mu_h/1. - self.mu_o) * expit (self.glad * (-grad + self.G/20)) + self.mu_o
            res = np.where(mu<=self.mu_o*1., 1, 0)
            res = np.where(mu_stop+res>=1, 1, 0)
            result = np.where(res==1, mu_new, mu)
        elif self.mu_type=='mu_run':
            result = mu
            res = None
        else:
            res = np.where(grad>grad_old, grad, grad_old)
            result = (self.mu_h-self.mu_o) * expit (self.glad * (-res + self.G)) + self.mu_o
        return result, res

    def lam_o(self, p, grad, grad_old, mu_stop, x):
        return self.k*self.rho_o(p, x) / self.mu_oil(grad, grad_old, mu_stop)[0]

    def coef_dp_dt(self, p, x):
        return self.m(p, x)*self.beta_o*self.rho_o0+self.rho_o(p, x)*self.beta_r

    def solution_init(self):
        p = np.zeros((self.nt, self.nx), dtype=np.float64)
        p[0, :] = self.p_0(self.x)
        # p[:,0] = self.p_left
        p[:,-1] = self.p_right
        return p

    def grad(self, p, dx, axis='center'):
        res = np.ones_like(p)
        if axis=='left':
            res[1:] = (p[1:]-p[:-1])/dx
            res[0] = (p[1]-p[0])/dx
        elif axis=='right':
            res[:-1] = (p[1:]-p[:-1])/dx
            res[-1] = (p[-1]-p[-2])/dx
        else:
            res = np.gradient(p, dx)
        return res

    def rate(self, t):
        n = self.pow_n
        a = self.Q*(self.T-self.t_0) / (self.tt**(n+1)/(n+1) + self.tt**n*(self.T-self.tt))
        return np.where(t<self.tt,a*t**(n),a*self.tt**(n))

    def residual(self, u_new, u_old, t_step, dt, dx):
        """Выражение невязки для метода Ньютона-Крылова."""
        N = len(u_new)
        res = np.zeros(N, dtype=np.float64)
        grad_right = self.grad(u_new, self.dx, axis='center')
        grad_left = self.grad(u_new, self.dx, axis='center')
        grad_old = self.grad_mu[-1]
        k_left_o = self.lam_o(u_new[1:-1], grad_left[1:-1], grad_old[1:-1], self.mu_stop[1:-1], self.x[1:-1])
        k_right_o = self.lam_o(u_new[2:], grad_right[2:], grad_old[2:], self.mu_stop[2:], self.x[2:])

        res[1:-1] = self.coef_dp_dt(u_new[1:-1], self.x[1:-1])*(u_new[1:-1] - u_old[1:-1]) / dt - \
                    (k_right_o*(u_new[2:] - u_new[1:-1]) / dx**2 - k_left_o*(u_new[1:-1] - u_new[:-2]) / dx**2)

        # Граничные условия
        if self.Q is None:
            res[0] = u_new[0] - self.p_left
        else:
            res[0] = (u_new[1] - u_new[0]) / dx * self.k / self.mu_oil(grad_left[0], grad_old[0], self.mu_stop[0])[0]*self.S + self.rate(t_step)
        res[-1] = u_new[-1] - self.p_right

        return res

    # Метод Ньютона-Крылова для решения нелинейной системы
    def solve_nonlinear(self, u_old, t_step, dt, dx):
        """Решение системы уравнений с использованием метода Ньютона-Крылова."""
        u_new_guess = np.copy(u_old)  # начальное предположение
        u_new = newton_krylov(lambda u_new: self.residual(u_new, u_old, t_step, dt, dx), u_new_guess, x_rtol=1e-4)
        return u_new

    def finite_diffs(self, C_skv=0):
        A, B, C, F = np.zeros(self.nx), np.zeros(self.nx), np.zeros(self.nx), np.zeros(self.nx)
        alfa, beta = np.zeros(self.nx), np.zeros(self.nx)
        P, Ps, Pn = np.zeros((self.nx, self.nt)), np.zeros(self.nx), np.zeros(self.nx)
        grad = np.zeros(self.nx)


        h = (self.L)/(self.nx-1)
        x = np.linspace(0, self.L, self.nx)

        tau = self.T / (self.nt - 1)
        t = np.linspace(0, self.T, self.nt)

        epsilon = 1.e-5

        P[:,0] = self.p_right

        def mu_oil(grad):
            return self.mu_o

        for j in range(1, self.nt):
            Pn = P[:,j-1].copy()
            y = 0
            while True:
                if y==0:
                    Ps = P[:,j-1].copy()
                else:
                    Ps = P[:,j].copy()
                y += 1

                for i in range(1,self.nx-1):
                    grad[i] = (Ps[i+1]-Ps[i-1])/(2*h)
                grad[0] = (Ps[1]-Ps[0])/(h)
                grad[-1] = (Ps[-1]-Ps[-2])/(h)


                alfa[0] = 1
                beta[0] = self.Q / (self.S*self.k/(mu_oil(grad[0])*h))


                for i in range(1, self.nx-1):
                    A[i] = 1. / h**2 * self.k / mu_oil(grad[i])
                    B[i] = 1. / h**2 * self.k / mu_oil(grad[i])
                    C[i] = A[i] + B[i] + self.beta_r/tau
                    F[i] = self.beta_r/tau*Pn[i]

                for i in range(1, self.nx-1):
                    alfa[i]=A[i]/(C[i]-B[i]*alfa[i-1])
                    beta[i]=(B[i]*beta[i-1]+F[i])/(C[i]-B[i]*alfa[i-1])

                P[-1][j] = self.p_right

                for i in range(self.nx-2, -1, -1):
                    P[i][j]=alfa[i]*P[i+1][j]+beta[i]

                u = abs((P[1][j]-Ps[1])/P[1][j])
                for i in range(self.nx):
                    if u < abs((P[i][j]-Ps[i])/P[i][j]):
                        u = abs((P[i][j]-Ps[i])/P[i][j])

                if u < epsilon:
                    break

        return P, x, t

    def solve(self):
        p = self.solution_init()
        mu_oil_init = np.zeros(self.nx, dtype=np.float64)+self.mu_h
        self.mu_oil_arr.append(mu_oil_init)
        self.grad_p.append(self.grad(p[0,:], self.dx))
        self.grad_mu.append(self.grad(p[0,:], self.dx))
        # Решение системы с использованием Ньютона-Крылова
        for t, t_step in enumerate(self.time[:-1]):
            # решение уравнения для давления
            p_old = p[t, :].copy()
            p_new = self.solve_nonlinear(p_old, t_step, self.dt, self.dx)
            p[t+1, :] = p_new.copy()
            grad_new = self.grad(p_new, self.dx)
            # print(expit (self.glad * (-self.grad_p[-1] + self.G)))
            mu_oil_new, res = self.mu_oil(grad_new, self.grad_mu[-1], self.mu_stop)
            self.mu_stop = res
            self.grad_mu.append(res)
            self.mu_oil_arr.append(mu_oil_new)
            self.grad_p.append(grad_new)
        self.mu_oil_arr = np.array(self.mu_oil_arr)
        self.grad_p = np.array(self.grad_p)
        return p

### Валидация с КР (линейная нефть)

$$\mu_h=2e-3$$
$$\beta_o=0$$
$$Q=5 м3/сут$$
$$p0=10e+6$$

In [27]:
solver1 = OnePhase(mu_h = 2e-3,
                mu_o = 2e-3,
                glad = 0.0002,
                G = 0.6e5,
                k = 5e-15,
                m0 = 0.2,
                beta_r = 1e-10,
                rho_o0 = 800.,
                beta_o = 5e-20, # посмотреть в таблице
                t_0 = 0,
                tt=0.001*1*86400,
                T = 1*86400,
                nt = 150*5,
                pow_n = 2,
                x_0 = 0,
                L = 100,
                L0 = 100,
                S = 50,
                nx = 200,
                p_left = 10e+6,
                p_right = 10e+6,
                mu_type = "mu_2sid",
                Q=5/86400)

p = solver1.solve()
p_fd, x_fd, t_xd = solver1.finite_diffs()
p_fd = p_fd.transpose()
G = -np.ones_like(solver1.grad_p)*solver1.G

In [ ]:
x1 = copy(solver1.x)
p1 = copy(p)
p_fd1 = copy(p_fd)
mu_oil_arr1 = copy(solver1.mu_oil_arr)
grad_p1 = copy(solver1.grad_p)


solution_plot([[x1, p1],
               [x_fd, p_fd1],
               [x1, mu_oil_arr1],
            #    [solver.x, abs(G)],
               [x1, abs(grad_p1)],
               [x1, -solver1.k/mu_oil_arr1*grad_p1]],
               titles=("Pressure (p) vs x", "mu_oil", "grad(p)", 'w_oil'),
               idx=[[1,1], [1,1], [1,2], [2,1], [2,2]],
               nt=solver1.nt,
               height=1000,
               width=1200)


### Нелинейная нефть, сравнение с линейной.

$$\mu_h=10e-3$$
$$\beta_o=0$$
$$Q=5 м3/сут$$
$$p0=10e+6$$

In [41]:
solver2 = OnePhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.0002,
                G = 0.6e5,
                k = 5e-15,
                m0 = 0.2,
                beta_r = 1e-10,
                rho_o0 = 800.,
                beta_o = 5e-20, # посмотреть в таблице
                t_0 = 0,
                tt=0.001*1*86400,
                T = 1*86400,
                nt = 150*5,
                pow_n = 2,
                x_0 = 0,
                L = 100,
                L0 = 100,
                S = 50,
                nx = 200,
                p_left = 10e+6,
                p_right = 10e+6,
                mu_type = "mu_2sid",
                Q=5/86400)

p = solver2.solve()
G = -np.ones_like(solver2.grad_p)*solver2.G

In [ ]:
x2 = copy(solver2.x)
p2 = copy(p)
mu_oil_arr2 = copy(solver2.mu_oil_arr)
grad_p2 = copy(solver2.grad_p)


solution_plot([[x2, p2],
               [x1, p1],
               [x1, mu_oil_arr2],
               [solver2.x, abs(G)],
               [x2, abs(grad_p2)],
               [x1, abs(grad_p1)],
               [x2, -solver2.k/mu_oil_arr2*grad_p2],
               [x1, -solver1.k/mu_oil_arr1*grad_p1]],
               titles=("Pressure (p) vs x", "mu_oil", "grad(p)", 'w_oil'),
               idx=[[1,1], [1,1], [1,2], [2,1], [2,1], [2,1], [2,2], [2,2]],
               nt=solver2.nt,
               height=1000,
               width=1200)


### Добавляем сжимаемость нефти. Сравнение с несжимаемой.

$$\mu_h=10e-3$$
$$\beta_o=5e-9$$
$$Q=5 м3/сут$$
$$p0=10e+6$$

In [ ]:
solver3 = OnePhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.0002,
                G = 0.6e5,
                k = 5e-15,
                m0 = 0.2,
                beta_r = 1e-10,
                rho_o0 = 800.,
                beta_o = 5e-9, # посмотреть в таблице
                t_0 = 0,
                tt=0.001*1*86400,
                T = 1*86400,
                nt = 150*5,
                pow_n = 2,
                x_0 = 0,
                L = 100,
                L0 = 100,
                S = 50,
                nx = 200,
                p_left = 10e+6,
                p_right = 10e+6,
                mu_type = "mu_2sid",
                Q=5/86400)

p = solver3.solve()
G = -np.ones_like(solver2.grad_p)*solver2.G

In [ ]:
x3 = copy(solver3.x)
p3 = copy(p)
mu_oil_arr3 = copy(solver3.mu_oil_arr)
grad_p3 = copy(solver3.grad_p)


solution_plot([[x3, p3],
               [x2, p2],
               [x3, mu_oil_arr3],
               [x2, mu_oil_arr2],
               [solver3.x, abs(G)],
               [x2, abs(grad_p2)],
               [x3, abs(grad_p3)],
               [x2, -solver2.k/mu_oil_arr2*grad_p2],
               [x3, -solver3.k/mu_oil_arr3*grad_p3]],
               titles=("Pressure (p) vs x", "mu_oil", "grad(p)", 'w_oil'),
               idx=[[1,1], [1,1], [1,2], [1,2], [2,1], [2,1], [2,1], [2,2], [2,2]],
               nt=solver2.nt,
               height=1000,
               width=1200)


### Анализ поведения на скачке

$$\mu_h=10e-3$$
$$glad=(0.0002, 0.0006)$$
$$Q=5 м3/сут$$
$$p0=10e+6$$

In [ ]:
solver4 = OnePhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.001,
                G = 0.6e5,
                k = 5e-15,
                m0 = 0.2,
                beta_r = 1e-10,
                rho_o0 = 800.,
                beta_o = 5e-9, # посмотреть в таблице
                t_0 = 0,
                tt=0.001*1*86400,
                T = 1*86400,
                nt = 150*5,
                pow_n = 2,
                x_0 = 0,
                L = 100,
                L0 = 100,
                S = 50,
                nx = 200,
                p_left = 10e+6,
                p_right = 10e+6,
                mu_type = "mu_2sid",
                Q=5/86400)

p = solver4.solve()
G = -np.ones_like(solver4.grad_p)*solver4.G

In [ ]:
x4 = copy(solver4.x)
p4 = copy(p)
mu_oil_arr4 = copy(solver4.mu_oil_arr)
grad_p4 = copy(solver4.grad_p)

grad_visc = np.linspace(0, 1.2e5, 200)
oil_visc3 = oil_visc(solver3.mu_h, solver3.mu_o, solver3.glad, grad_visc, solver3.G)
oil_visc4 = oil_visc(solver4.mu_h, solver4.mu_o, solver4.glad, grad_visc, solver4.G)

solution_plot([[x3, p3],
               [x4, p4],
               [x3, mu_oil_arr3],
               [x4, mu_oil_arr4],
               [solver4.x, abs(G)],
               [x4, abs(grad_p4)],
               [x3, abs(grad_p3)],
               [x4, -solver4.k/mu_oil_arr4*grad_p4],
               [x3, -solver3.k/mu_oil_arr3*grad_p3]],
               titles=("Pressure (p) vs x", "mu_oil", "grad(p)", 'w_oil'),
               idx=[[1,1], [1,1], [1,2], [1,2], [2,1], [2,1], [2,1], [2,2], [2,2]],
               nt=solver2.nt,
               height=1000,
               width=1200)

plt.plot(grad_visc, oil_visc4, grad_visc, oil_visc3)
plt.show()
# plt.plot(solver4.time, abs(grad_p4[:,30]))
# plt.show()

### Другое ГУ

$$\mu_h=10e-3$$
$$p_{left}=12e+6$$
$$p0=10e+6$$

In [ ]:
solver5 = OnePhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.0002,
                G = 0.6e5,
                k = 5e-15,
                m0 = 0.2,
                beta_r = 1e-10,
                rho_o0 = 800.,
                beta_o = 5e-9, # посмотреть в таблице
                t_0 = 0,
                tt=0.001*1*86400,
                T = 5*86400,
                nt = 150*5,
                pow_n = 2,
                x_0 = 0,
                L = 100,
                L0 = 500,
                S = 50,
                nx = 200,
                p_left = 12e+6,
                p_right = 10e+6,
                mu_type = "mu",
                Q=None)

p = solver5.solve()
G = -np.ones_like(solver5.grad_p)*solver5.G

In [ ]:
x5 = copy(solver5.x)
p5 = copy(p)
mu_oil_arr5 = copy(solver5.mu_oil_arr)
grad_p5 = copy(solver5.grad_p)


solution_plot([
               [x5, p5],
               [x5, mu_oil_arr5],
               [solver5.x, abs(G)],
               [x5, abs(grad_p5)],
               [x5, -solver5.k/mu_oil_arr5*grad_p5]],
               titles=("Pressure (p) vs x", "mu_oil", "grad(p)", 'w_oil'),
               idx=[[1,1], [1,2], [2,1], [2,1], [2,2]],
               nt=solver2.nt,
               height=1000,
               width=1200)

### Другое ГУ

$$\mu_h=10e-3$$
$$p_{left}=12e+6$$
$$p_{right}=10e+6$$
$$L0=L/2$$

In [91]:
solver6 = OnePhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.0002,
                G = 0.6e5,
                k = 5e-15,
                m0 = 0.2,
                beta_r = 1e-10,
                rho_o0 = 800.,
                beta_o = 5e-9, # посмотреть в таблице
                t_0 = 0,
                tt=0.001*1*86400,
                T = 5*86400,
                nt = 150*5,
                pow_n = 2,
                x_0 = 0,
                L = 100,
                L0 = 2,
                S = 50,
                nx = 200,
                p_left = 12e+6,
                p_right = 10e+6,
                mu_type = "mu",
                Q=None)

p = solver6.solve()
G = -np.ones_like(solver5.grad_p)*solver6.G

In [ ]:
x6 = copy(solver6.x)
p6 = copy(p)
mu_oil_arr6 = copy(solver6.mu_oil_arr)
grad_p6 = copy(solver6.grad_p)


solution_plot([
               [x6, p6],
               [x6, mu_oil_arr6],
               [solver6.x, abs(G)],
               [x6, abs(grad_p6)],
               [x6, -solver6.k/mu_oil_arr6*grad_p6]],
               titles=("Pressure (p) vs x", "mu_oil", "grad(p)", 'w_oil'),
               idx=[[1,1], [1,2], [2,1], [2,1], [2,2]],
               nt=solver6.nt,
               height=1000,
               width=1200)

### Другое ГУ

$$\mu_h=10e-3$$
$$Q=5m3/c$$
$$p_{left}=12e+6$$
$$p_{right}=10e+6$$
$$L0=L/2$$

In [94]:
solver7 = OnePhase(mu_h = 10e-3,
                mu_o = 2e-3,
                glad = 0.0002,
                G = 0.6e5,
                k = 5e-15,
                m0 = 0.2,
                beta_r = 1e-10,
                rho_o0 = 800.,
                beta_o = 5e-9, # посмотреть в таблице
                t_0 = 0,
                tt=0.01*1*86400,
                T = 5*86400,
                nt = 150*5,
                pow_n = 2,
                x_0 = 0,
                L = 100,
                L0 = 2,
                S = 50,
                nx = 200,
                p_left = 12e+6,
                p_right = 10e+6,
                mu_type = "mu",
                Q=5/86400)

p = solver7.solve()
G = -np.ones_like(solver7.grad_p)*solver7.G

In [ ]:
x7 = copy(solver7.x)
p7 = copy(p)
mu_oil_arr7 = copy(solver7.mu_oil_arr)
grad_p7 = copy(solver7.grad_p)


solution_plot([
               [x7, p7],
               [x7, mu_oil_arr7],
               [solver7.x, abs(G)],
               [x7, abs(grad_p7)],
               [x7, -solver6.k/mu_oil_arr7*grad_p7]],
               titles=("Pressure (p) vs x", "mu_oil", "grad(p)", 'w_oil'),
               idx=[[1,1], [1,2], [2,1], [2,1], [2,2]],
               nt=solver7.nt,
               height=1000,
               width=1200)